In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms

import time
import numpy as np
import i8ie

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(784, 10)
    def forward(self, x):
        x = x.reshape(-1, 784)
        x = self.fc(x)
        return x
    
class MyNet(i8ie.Module):
    def __init__(self):
        super().__init__()
        self.fc = i8ie.Linear(784, 10)
    def forward(self, x):
        x = x.reshape(-1, 784)
        x = self.fc(x)
        return x
    
state_dict = torch.load('fc_mnist28.pt')
torch_model = Net()
torch_model.load_state_dict(state_dict)
my_model = MyNet()
my_model.load(state_dict)

In [2]:
batch_size = 100
transform = torchvision.transforms.Compose(
    [transforms.ToTensor()]
)

test_dataset = torchvision.datasets.MNIST('./data/mnist', train=False, download=True,transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
cal_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=True)

xs = []
ts = []
xss = []
tss = []
for batch_idx, (x, target) in enumerate(test_loader):
    xs.append(i8ie.tensor(x))
    ts.append(i8ie.tensor(target))
    xss.append(x)
    tss.append(target)
    if batch_idx >= 100:
        break

In [3]:
%%time
correct_cnt = 0
for x,target in zip(xss,tss):
    out = torch_model(x)
    _, pred_label = torch.max(out.data, 1)
    correct_cnt += (pred_label == target.data).sum()
    
print(correct_cnt)

tensor(9250)
CPU times: user 63 ms, sys: 0 ns, total: 63 ms
Wall time: 9 ms


In [4]:
%%time
correct_cnt = 0
for x,target in zip(xs,ts):
    x = my_model(x)
    p = i8ie.argmax(x, axis = 1)
    correct_cnt += (p == target).sum()

print(correct_cnt)

9250.0
CPU times: user 30.2 ms, sys: 0 ns, total: 30.2 ms
Wall time: 3.76 ms


In [5]:
%%time
my_model.prepare()
for batch_idx, (x, target) in enumerate(cal_loader):
    x = my_model(i8ie.tensor(x))
    break
my_model.convert()

CPU times: user 55.5 ms, sys: 3.83 ms, total: 59.3 ms
Wall time: 7.39 ms


In [6]:
%%time
correct_cnt = 0
for x,target in zip(xs,ts):
    x = my_model(x)
    p = i8ie.argmax(x, axis = 1)
    correct_cnt += (p == target).sum()
print(correct_cnt)

9256.0
CPU times: user 156 ms, sys: 0 ns, total: 156 ms
Wall time: 19.6 ms
